<a href="https://colab.research.google.com/github/andrePankraz/qa_service/blob/main/notebooks/Elterngeld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elterngeld Digital FAQ-Suche
Install necessary packages.

In [37]:
!pip install --quiet openai stanza tiktoken sentence-transformers

Read FAQ web page, extract text data and export as JSON lines file 'faq.jsonl'.

In [38]:
import requests
from bs4 import BeautifulSoup
import re
import json

def extract_elterngeld_digital_faq():
  url = 'https://www.elterngeld-digital.de/ams/content/Elterngeld/haeufige-fragen-elterngeld'
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  faq = [["ID", "Thema_ID", "Thema", "Frage_ID", "Frage", "Antwort"]]

  main_form_element = soup.find('form', {'id': 'mainForm'})
  for id1, faq_element in enumerate(main_form_element.select("h3[id^=faq]")):
    topic_id = faq_element['id']
    topic = ''
    for child_elem in faq_element.contents:
      if child_elem.name == None:
        topic += child_elem
      elif child_elem.name == 'span':
        topic += " (" + child_elem.contents[0] + ")"
    for id2, details_element in enumerate(faq_element.find_next_siblings('details')):
      id = str(id1 + 1) + '.' + str(id2 + 1)
      question_id = details_element['id']
      question = ''
      response = ''
      for child_elem in details_element.contents:
        if child_elem.name == None:
          response += child_elem
        elif child_elem.name == 'summary':
          question = child_elem.get_text()
        else:
          response += child_elem.get_text()
      response = re.sub('[\n\r ]+', ' ', response).strip()
      faq.append([id, topic_id, topic, question_id, question, response])

  with open('faq.jsonl', 'w', encoding="utf-8") as f:
      for entry in faq:
          json.dump(entry, f, ensure_ascii=False)
          f.write('\n')

extract_elterngeld_digital_faq()

Import JSON lines file 'faq.jsonl' and convert into generic format for document question answering.

In [39]:
def import_elterngeld_documents() -> list[tuple[str, str, str]]:
    faq = []
    with open("faq.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            faq.append(obj)

    # ID, Title, Text
    documents = [(f[0], f[4], f[5]) for f in faq[1:]]
    return documents

documents = import_elterngeld_documents()
documents

[('1.1',
  'Welche Voraussetzungen muss ich für Elterngeld erfüllen?',
  'Für das Elterngeld müssen Sie folgende Voraussetzungen erfüllen: Sie betreuen und erziehen Ihr Kind selbst, Sie haben einen Wohnsitz in Deutschland oder Sie halten sich gewöhnlich hier auf und Sie leben mit Ihrem Kind in einem gemeinsamen Haushalt. \xa0Für Geburten ab dem 1. September 2021 außerdem: Sie sind entweder gar nicht erwerbstätig oder arbeiten nicht mehr als 32 Stunden pro Woche und Ihr zu versteuerndes jährliches Einkommen vor der Geburt lag nicht über 250.000 Euro. Für Elternpaare und getrennt Erziehende darf das Einkommen, das sie gemeinsam versteuern, nicht über 300.000 Euro liegen. Für Geburten bis zum 31. August 2021 außerdem: Sie sind entweder gar nicht erwerbstätig oder arbeiten nicht mehr als 30 Wochenstunden im Durchschnitt des Lebensmonats und Ihr zu versteuerndes jährliches Einkommen vor der Geburt lag nicht über 250.000 Euro. Für Elternpaare darf das Einkommen, das sie gemeinsam versteuern,

Import on-pemise embedding model.

In [40]:
import torch
from sentence_transformers import SentenceTransformer, util

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# OKish models for german (and sometimes also english)...
# German only, quite good and small:
# embedding_model_id = 'PM-AI/bi-encoder_msmarco_bert-base_german'
# Bilingual, quite good and small
# See good Benchmarks: https://huggingface.co/PM-AI/sts_paraphrase_xlm-roberta-base_de-en
# embedding_model_id = 'PM-AI/sts_paraphrase_xlm-roberta-base_de-en' # prefers cos
# German only, OKish and midsized:
# embedding_model_id = 'Sahajtomar/German-semantic'
# Multilingual, pretty good and small
# MPNet seems to be better than RoBERTa variantes, even though short 128er windows
embedding_model_id = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

embedding_model = SentenceTransformer(embedding_model_id, device=device)
embedding_max_seq_length = embedding_model.max_seq_length

embedding_model.device, embedding_model, embedding_max_seq_length

(device(type='cpu'),
 SentenceTransformer(
   (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
 ),
 128)

Split documents into paragraphs for embedding.

In [41]:
import stanza
import tiktoken

def split_into_paragraphs(documents, max_sentences, overlap_sentences, max_tokens, overlap_tokens):
  paragraphs = []  # resulting list

  # configure stanza for sentence splitting in multiple languages
  nlp = stanza.MultilingualPipeline(
      lang_id_config={"langid_clean_text": True},
      lang_configs={'de': {'processors': 'tokenize,mwt'}, 'en': {'processors': 'tokenize'}})
  # configure tiktoken for token splitting in target embedding model
  embedding_encoding = tiktoken.encoding_for_model("text-embedding-ada-002")

  for nr,document in enumerate(documents):
    id, title, text = document
    # pre-calculate token number for title (part of embedding paragraph)
    title_tokens = len(embedding_encoding.encode(title))
    # split text into sentences
    nlp_sentences = nlp(text).sentences
    # pre-calculate token numbers for each sentence
    sentence_tokens = [len(embedding_encoding.encode(nlp_sentence.text)) for nlp_sentence in nlp_sentences]

    sentence_index = -1
    paragraph = ''
    paragraph_sentences = 0
    paragraph_tokens = 0

    index = 0
    while index < len(sentence_tokens):
      tokens = sentence_tokens[index]

      if sentence_index == -1:
        # start new paragraph
        sentence_index = index
        paragraph = nlp_sentences[index].text
        paragraph_sentences = 1
        paragraph_tokens = title_tokens + 1 + tokens
        index += 1
        continue

      if (max_sentences <= 0 or paragraph_sentences < max_sentences) and (max_tokens <= 0 or paragraph_tokens + tokens <= max_tokens):
        # continue paragraph
        paragraph += ' ' + nlp_sentences[index].text
        paragraph_sentences += 1
        paragraph_tokens += 1 + tokens
        index += 1
        continue

      # finish paragraph
      paragraphs.append((nr, sentence_index, title, paragraph))

      # overlap paragraphs with sentence or token window - whatever boundary triggered first
      if max_sentences > 0 and paragraph_sentences == max_sentences and overlap_sentences <= max_sentences / 2:
        index -= overlap_sentences
      if max_tokens > 0 and paragraph_tokens + tokens > max_tokens and overlap_tokens > 0 and overlap_tokens <= max_tokens / 2:
        overlap_tokens_sum = 0
        while index > sentence_index + 1:
          overlap_tokens_sum += sentence_tokens[index - 1]
          if overlap_tokens_sum > overlap_tokens:
            break
          index -= 1

      # trigger new paragraph
      sentence_index = -1
    else:
      if sentence_index != -1:
        # add final paragraph
        paragraphs.append((nr, sentence_index, title, paragraph))
  return paragraphs


max_sentences = 6
overlap_sentences = 1
max_tokens = embedding_max_seq_length
overlap_tokens = max_tokens / 6

paragraphs = split_into_paragraphs(documents, max_sentences, overlap_sentences, max_tokens, overlap_tokens)

paragraphs, len(paragraphs), max_tokens

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: multilingual ():
| Processor | Package |
-----------------------
| langid    | ud      |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: langid
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!


([(0,
   0,
   'Welche Voraussetzungen muss ich für Elterngeld erfüllen?',
   'Für das Elterngeld müssen Sie folgende Voraussetzungen erfüllen: Sie betreuen und erziehen Ihr Kind selbst, Sie haben einen Wohnsitz in Deutschland oder Sie halten sich gewöhnlich hier auf und Sie leben mit Ihrem Kind in einem gemeinsamen Haushalt. Für Geburten ab dem 1. September 2021 außerdem:'),
  (0,
   2,
   'Welche Voraussetzungen muss ich für Elterngeld erfüllen?',
   'Für Geburten ab dem 1. September 2021 außerdem: Sie sind entweder gar nicht erwerbstätig oder arbeiten nicht mehr als 32 Stunden pro Woche und Ihr zu versteuerndes jährliches Einkommen vor der Geburt lag nicht über 250.000 Euro. Für Elternpaare und getrennt Erziehende darf das Einkommen, das sie gemeinsam versteuern, nicht über 300.000 Euro liegen.'),
  (0,
   5,
   'Welche Voraussetzungen muss ich für Elterngeld erfüllen?',
   'Für Geburten bis zum 31. August 2021 außerdem: Sie sind entweder gar nicht erwerbstätig oder arbeiten nicht m

Create embeddings with on-premise model. Prefix the paragraphs with the title, if title isn't already included into the paragraph.

In [42]:
embedding_paragraphs = [p[3] if p[2] in p[3] else p[2] + ': ' + p[3] for p in paragraphs]

embeddings_onprem = embedding_model.encode(embedding_paragraphs)

print(len(embeddings_onprem))

180


Find most relevant answers for query with on-premise model.

In [43]:
question = 'Welche Regelungen gelten für die Berechnung des Elterngelds bei Frühgeborene? '

query_embedding_onprem = embedding_model.encode(question)

sim_scores_onprem = util.cos_sim(query_embedding_onprem, embeddings_onprem)
# sim_scores_onprem = util.dot_score(query_embedding_onprem, embeddings_onprem)
top_results_onprem = torch.topk(sim_scores_onprem[0], k=30)

for score, idx in zip(top_results_onprem[0], top_results_onprem[1]):
  print(f"(Score: {score:.4f})  {paragraphs[idx]}")

(Score: 0.7791)  (13, 1, 'Wie wirken sich Mutterschaftsgeld und andere Mutterschaftsleistungen auf das Elterngeld aus?', 'Mutterschaftsleistungen erhalten Mütter zum Ausgleich für ihren Verdienstausfall in der Zeit, die sie direkt vor und nach der Geburt ihres Kindes nicht arbeiten dürfen.')
(Score: 0.7755)  (10, 60, 'Wie hoch ist das Elterngeld?', 'Im Jahr vor der Geburt müssen die Einkünfte also insgesamt unter 420 Euro geblieben sein; im Jahr der Geburt werden nur die Monate vor dem Geburtsmonat berücksichtigt. Wenn Sie im Veranlagungszeitraum zum Beispiel in Mutterschutz waren oder Elterngeld für ein älteres Kind bekommen haben, dann können Sie beantragen, dass der Bemessungszeitraum auf den Veranlagungszeitraum davor verschoben wird.')
(Score: 0.7743)  (8, 16, 'Was ist ElterngeldPlus?', 'Dann kann der Bezug – je nach Zeitpunkt der Geburt - auch nach dem 15. Lebensmonat unterbrochen werden. Weitere Details hierzu finden Sie unter dem Punkt "Welche Besonderheiten gelten bei Frühchen

In [44]:
def longest_overlap_suffix_prefix(s1, s2):
    for i in range(len(s1)-1, -1, -1):
        if s2.startswith(s1[i:]):
            return s1[i:]
    return ""

def merge_overlapping_fragments(fragments):
    result = fragments[0]
    for i in range(1, len(fragments)):
        overlap = longest_overlap_suffix_prefix(result, fragments[i])
        result += fragments[i][len(overlap):]
    return result

def restruct_top_paragraphs(nested_list):
    clustered_items = {}

    for item in nested_list:
        if item[0] not in clustered_items:
            clustered_items[item[0]] = []
        clustered_items[item[0]].append(item)

    # Sort each cluster by the second entry
    for key in clustered_items:
        clustered_items[key] = sorted(clustered_items[key], key=lambda x: x[1])

    # Combine clusters in the original order
    reordered_list = []
    for key in clustered_items:
        reordered_list.append([key, clustered_items[key][0][2], merge_overlapping_fragments([item[3] for item in clustered_items[key]])])

    return reordered_list

top_paragraphs = [paragraphs[idx] for idx in top_results_onprem[1]]
top_paragraphs = restruct_top_paragraphs(top_paragraphs)

top_paragraphs

[[13,
  'Wie wirken sich Mutterschaftsgeld und andere Mutterschaftsleistungen auf das Elterngeld aus?',
  'Mutterschaftsleistungen erhalten Mütter zum Ausgleich für ihren Verdienstausfall in der Zeit, die sie direkt vor und nach der Geburt ihres Kindes nicht arbeiten dürfen.Kein ElterngeldPlus oder Partnerschaftsbonus möglich Lebensmonate, in denen der Mutter des Kindes Mutterschaftsleistungen zustehen, gelten bei ihr als Monate mit Basiselterngeld. Für diese Monate kann sie also kein ElterngeldPlus und keinen Partnerschaftsbonus beantragen.Das Elterngeld hat denselben Zweck. Daher werden die Mutterschaftsleistungen auf das Elterngeld angerechnet, damit das weggefallene Einkommen nicht doppelt ausgeglichen wird. Mutterschaftsleistungen werden anders als das Elterngeld in Tagen berechnet und deshalb auch tageweise auf das Elterngeld angerechnet.Wenn in einem Lebensmonat noch Tage ohne Mutterschaftsleistungen übrig sind, kann es sich lohnen, Basiselterngeld für diesen Lebensmonat zu bean

In [45]:
import tiktoken

def top_facts(top_paragraphs, max_facts_tokens):
  facts = ''
  facts_tokens = 0
  prompt_encoding = tiktoken.encoding_for_model("gpt-4")

  for top_paragraph in top_paragraphs:
    fact = '[' + str(top_paragraph[0]) + '] ' + top_paragraph[1] + ' ' + top_paragraph[2]
    fact_tokens = len(prompt_encoding.encode(fact))
    if facts_tokens + fact_tokens > max_facts_tokens:
      break
    facts += '\n' + fact
    facts_tokens += 1 + fact_tokens

  return facts

facts = top_facts(top_paragraphs, 2000)

print(facts)


[13] Wie wirken sich Mutterschaftsgeld und andere Mutterschaftsleistungen auf das Elterngeld aus? Mutterschaftsleistungen erhalten Mütter zum Ausgleich für ihren Verdienstausfall in der Zeit, die sie direkt vor und nach der Geburt ihres Kindes nicht arbeiten dürfen.Kein ElterngeldPlus oder Partnerschaftsbonus möglich Lebensmonate, in denen der Mutter des Kindes Mutterschaftsleistungen zustehen, gelten bei ihr als Monate mit Basiselterngeld. Für diese Monate kann sie also kein ElterngeldPlus und keinen Partnerschaftsbonus beantragen.Das Elterngeld hat denselben Zweck. Daher werden die Mutterschaftsleistungen auf das Elterngeld angerechnet, damit das weggefallene Einkommen nicht doppelt ausgeglichen wird. Mutterschaftsleistungen werden anders als das Elterngeld in Tagen berechnet und deshalb auch tageweise auf das Elterngeld angerechnet.Wenn in einem Lebensmonat noch Tage ohne Mutterschaftsleistungen übrig sind, kann es sich lohnen, Basiselterngeld für diesen Lebensmonat zu beantragen. 

Load a question answering model.

In [46]:
import torch
from transformers import pipeline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

qa_model_id = 'deepset/gelectra-large-germanquad'
# qa_model_id = 'imvladikon/de_electra_squad'
qa_pipeline = pipeline('question-answering', model=qa_model_id, tokenizer=qa_model_id, device=device)

qa_pipeline.device

device(type='cuda', index=0)

Find answers with QA model. These QA models find the most relevant text fragments for the given question (as citation or reference) and cannot really do reasoning or create answers.

In [47]:
qa_pipeline({
    'question': question,
    'context': facts
}, top_k=5)

[{'score': 0.0007217896054498851,
  'start': 2566,
  'end': 2648,
  'answer': 'Wenn Ihr Kind mindestens 6 Wochen vor dem errechneten Geburtstermin zur Welt kommt'},
 {'score': 0.0006847308250144124,
  'start': 575,
  'end': 646,
  'answer': 'Daher werden die Mutterschaftsleistungen auf das Elterngeld angerechnet'},
 {'score': 0.0006127841188572347,
  'start': 2566,
  'end': 2570,
  'answer': 'Wenn'},
 {'score': 0.00044819674803875387,
  'start': 2650,
  'end': 2688,
  'answer': 'können Sie länger Elterngeld bekommen.'},
 {'score': 0.00038941463571973145,
  'start': 714,
  'end': 779,
  'answer': 'Mutterschaftsleistungen werden anders als das Elterngeld in Tagen'}]

# With OpenAI Services
Import Open API key.

In [48]:
from google.colab import drive
import json
import openai

drive.mount('/content/drive')
with open('/content/drive/My Drive/Private/api_keys.json', 'r') as f:
    api_keys = json.load(f)
openai.api_key = api_keys['openai']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Open AI Embedding Model
OpenAI model embedding model has huge context window size of 8191 tokens. Related paragraphs will mostly not be split.

In [49]:
max_sentences = 0
overlap_sentences = 0
max_tokens = 8191
overlap_tokens = max_tokens / 6

paragraphs = split_into_paragraphs(documents, max_sentences, overlap_sentences, max_tokens, overlap_tokens)
embedding_paragraphs = [p[3] if p[2] in p[3] else p[2] + ': ' + p[3] for p in paragraphs]

paragraphs, len(paragraphs), max_tokens

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: multilingual ():
| Processor | Package |
-----------------------
| langid    | ud      |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: langid
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!


([(0,
   0,
   'Welche Voraussetzungen muss ich für Elterngeld erfüllen?',
   'Für das Elterngeld müssen Sie folgende Voraussetzungen erfüllen: Sie betreuen und erziehen Ihr Kind selbst, Sie haben einen Wohnsitz in Deutschland oder Sie halten sich gewöhnlich hier auf und Sie leben mit Ihrem Kind in einem gemeinsamen Haushalt. Für Geburten ab dem 1. September 2021 außerdem: Sie sind entweder gar nicht erwerbstätig oder arbeiten nicht mehr als 32 Stunden pro Woche und Ihr zu versteuerndes jährliches Einkommen vor der Geburt lag nicht über 250.000 Euro. Für Elternpaare und getrennt Erziehende darf das Einkommen, das sie gemeinsam versteuern, nicht über 300.000 Euro liegen. Für Geburten bis zum 31. August 2021 außerdem: Sie sind entweder gar nicht erwerbstätig oder arbeiten nicht mehr als 30 Wochenstunden im Durchschnitt des Lebensmonats und Ihr zu versteuerndes jährliches Einkommen vor der Geburt lag nicht über 250.000 Euro. Für Elternpaare darf das Einkommen, das sie gemeinsam versteuern

Create embeddings with OpenAI model.

In [50]:
response = openai.Embedding.create(model='text-embedding-ada-002', input=embedding_paragraphs)

In [51]:
embeddings_openai = [e['embedding'] for e in response['data']]

Find most relevant answers for query with OpenAI model:

In [52]:
from sentence_transformers import util
import torch

query_embedding_openai = openai.Embedding.create(
    model='text-embedding-ada-002',
    input=question)['data'][0]['embedding']

sim_scores_openai = util.cos_sim(query_embedding_openai, embeddings_openai)
top_results_openai = torch.topk(sim_scores_openai[0], k=10)

for score, idx in zip(top_results_openai[0], top_results_openai[1]):
  print(f"(Score: {score:.4f})  {paragraphs[idx]}")

(Score: 0.8947)  (4, 0, 'Welche Besonderheiten gelten bei Frühchen?', 'Wenn Ihr Kind mindestens 6 Wochen vor dem errechneten Geburtstermin zur Welt kommt, können Sie länger Elterngeld bekommen. Dieser längere Bezug ist nur möglich, wenn das Kind ab dem 1. September 2021 geboren ist. Bis zu 4 zusätzliche Monate Basiselterngeld sind möglich, abhängig vom Geburtstermin: bei einer Geburt mindestens 6 Wochen vor dem errechneten Termin: 1 zusätzlicher Monat Basiselterngeld bei einer Geburt mindestens 8 Wochen vor dem errechneten Termin: 2 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 12 Wochen vor dem errechneten Termin: 3 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 16 Wochen vor dem errechneten Termin: 4 zusätzliche Monate Basiselterngeld Wie sonst auch können Sie jeden dieser zusätzlichen Monate mit Basiselterngeld tauschen gegen jeweils 2 Monate mit ElterngeldPlus. Für diese zusätzlichen Monate werden auch Ihre Gestaltungs-Möglichkeiten erweitert: Bei e

In [53]:
top_paragraphs = [paragraphs[idx] for idx in top_results_openai[1]]
top_paragraphs = restruct_top_paragraphs(top_paragraphs)

top_paragraphs

[[4,
  'Welche Besonderheiten gelten bei Frühchen?',
  'Wenn Ihr Kind mindestens 6 Wochen vor dem errechneten Geburtstermin zur Welt kommt, können Sie länger Elterngeld bekommen. Dieser längere Bezug ist nur möglich, wenn das Kind ab dem 1. September 2021 geboren ist. Bis zu 4 zusätzliche Monate Basiselterngeld sind möglich, abhängig vom Geburtstermin: bei einer Geburt mindestens 6 Wochen vor dem errechneten Termin: 1 zusätzlicher Monat Basiselterngeld bei einer Geburt mindestens 8 Wochen vor dem errechneten Termin: 2 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 12 Wochen vor dem errechneten Termin: 3 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 16 Wochen vor dem errechneten Termin: 4 zusätzliche Monate Basiselterngeld Wie sonst auch können Sie jeden dieser zusätzlichen Monate mit Basiselterngeld tauschen gegen jeweils 2 Monate mit ElterngeldPlus. Für diese zusätzlichen Monate werden auch Ihre Gestaltungs-Möglichkeiten erweitert: Bei einem zusätzlich

In [54]:
facts = top_facts(top_paragraphs, 2000)

print(facts)


[4] Welche Besonderheiten gelten bei Frühchen? Wenn Ihr Kind mindestens 6 Wochen vor dem errechneten Geburtstermin zur Welt kommt, können Sie länger Elterngeld bekommen. Dieser längere Bezug ist nur möglich, wenn das Kind ab dem 1. September 2021 geboren ist. Bis zu 4 zusätzliche Monate Basiselterngeld sind möglich, abhängig vom Geburtstermin: bei einer Geburt mindestens 6 Wochen vor dem errechneten Termin: 1 zusätzlicher Monat Basiselterngeld bei einer Geburt mindestens 8 Wochen vor dem errechneten Termin: 2 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 12 Wochen vor dem errechneten Termin: 3 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 16 Wochen vor dem errechneten Termin: 4 zusätzliche Monate Basiselterngeld Wie sonst auch können Sie jeden dieser zusätzlichen Monate mit Basiselterngeld tauschen gegen jeweils 2 Monate mit ElterngeldPlus. Für diese zusätzlichen Monate werden auch Ihre Gestaltungs-Möglichkeiten erweitert: Bei einem zusätzlichen Monat

## Open AI Generative Language Model
Generate prompt for generative language model.

In [55]:
prompt = """Du bist eine herausragende Suchmaschine, die in einem Webportal natürlichsprachige Fragen beantwortet.
Im Prompt folgen die Frage und mehrere Fakten, die möglicherweise zur Frage passen.
Die einzelnen Fakten beginnen jeweils mit Referenzangabe [x].
Nutze für die Antwort ausschließlich den einen Fakt, der besonders relevant für die gestellte Frage ist und starte mit der Referenzangabe [x].
Wenn nichts passt, erfinde keine Fakten, sondern antworte mit: 'Ich bin mit nicht sicher.'
Antworte kurz und prägnant in maximal 250 Wörtern.
Formuliere in verständlichen Sätzen und nutze einfache Sprache.
Denke Schritt für Schritt und prüfe Deine Antwort.

Frage: """ + question + '\n\nFakten:\n-----' + facts + '\n-----\nAntwort:'
print(prompt)

Du bist eine herausragende Suchmaschine, die in einem Webportal natürlichsprachige Fragen beantwortet.
Im Prompt folgen die Frage und mehrere Fakten, die möglicherweise zur Frage passen.
Die einzelnen Fakten beginnen jeweils mit Referenzangabe [x].
Nutze für die Antwort ausschließlich den einen Fakt, der besonders relevant für die gestellte Frage ist und starte mit der Referenzangabe [x].
Wenn nichts passt, erfinde keine Fakten, sondern antworte mit: 'Ich bin mit nicht sicher.'
Antworte kurz und prägnant in maximal 250 Wörtern.
Formuliere in verständlichen Sätzen und nutze einfache Sprache.
Denke Schritt für Schritt und prüfe Deine Antwort.

Frage: Welche Regelungen gelten für die Berechnung des Elterngelds bei Frühgeborene? 

Fakten:
-----
[4] Welche Besonderheiten gelten bei Frühchen? Wenn Ihr Kind mindestens 6 Wochen vor dem errechneten Geburtstermin zur Welt kommt, können Sie länger Elterngeld bekommen. Dieser längere Bezug ist nur möglich, wenn das Kind ab dem 1. September 2021 ge

Call generative language model.

In [56]:
response = openai.Completion.create(model="text-davinci-003", prompt=prompt, max_tokens=500)
response

<OpenAIObject text_completion id=cmpl-76zIF4qVGMLCfAhkVor1q9wChDsge at 0x7f6dbfd831d0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " [4] Wenn Ihr Kind mindestens 6 Wochen vor dem errechneten Geburtstermin zur Welt kommt, k\u00f6nnen Sie l\u00e4nger Elterngeld bekommen. Bis zu 4 zus\u00e4tzliche Monate Basiselterngeld sind m\u00f6glich, abh\u00e4ngig vom Geburtstermin: bei einer Geburt mindestens 6 Wochen vor dem errechneten Termin: 1 zus\u00e4tzlicher Monat Basiselterngeld bei einer Geburt mindestens 8 Wochen vor dem errechneten Termin: 2 zus\u00e4tzliche Monate Basiselterngeld bei einer Geburt mindestens 12 Wochen vor dem errechneten Termin: 3 zus\u00e4tzliche Monate Basiselterngeld bei einer Geburt mindestens 16 Wochen vor dem errechneten Termin: 4 zus\u00e4tzliche Monate Basiselterngeld. Es besteht auch die M\u00f6glichkeit, jeden dieser zus\u00e4tzlichen Monate mit Basiselterngeld gegen jeweils 2 Monate Elterng

Extract response text.

In [57]:
response['choices'][0]['text'].strip()

'[4] Wenn Ihr Kind mindestens 6 Wochen vor dem errechneten Geburtstermin zur Welt kommt, können Sie länger Elterngeld bekommen. Bis zu 4 zusätzliche Monate Basiselterngeld sind möglich, abhängig vom Geburtstermin: bei einer Geburt mindestens 6 Wochen vor dem errechneten Termin: 1 zusätzlicher Monat Basiselterngeld bei einer Geburt mindestens 8 Wochen vor dem errechneten Termin: 2 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 12 Wochen vor dem errechneten Termin: 3 zusätzliche Monate Basiselterngeld bei einer Geburt mindestens 16 Wochen vor dem errechneten Termin: 4 zusätzliche Monate Basiselterngeld. Es besteht auch die Möglichkeit, jeden dieser zusätzlichen Monate mit Basiselterngeld gegen jeweils 2 Monate ElterngeldPlus zu tauschen. Ihr Elterngeld wird auf Basis des Einkommens im Bemessungszeitraum berechnet. Für systemrelevante Berufe gab es zudem eine Sonderregelung; diese ist aber mittlerweile ausgelaufen.'